This code is a C++ implementation of a class called "RosbotClass" that uses ROS (Robot Operating System) to control a robot. The class contains methods for moving the robot forward and backward, turning clockwise and counterclockwise, stopping the robot's movement, and getting its position.

The constructor initializes the ROS node and subscribes to the laser and odometry topics. The laser callback function stores the laser range in a member variable. The odom callback function stores the x, y, and z position of the robot in member variables.

The move, move_forward, move_backwards, and turn functions all use a while loop to repeatedly publish velocity commands to the robot. The stop_moving function simply sets the velocity commands to zero.

The get_position and get_position_full functions return the x, y, and z position of the robot. The get_time function returns the current time in seconds.

Overall, this code provides a basic implementation for controlling a robot using ROS. However, it is important to note that it may not be suitable for all robots or environments and may require modification to work properly.

In [ ]:
#include "rosbot_control/rosbot_class.h"
#include "geometry_msgs/Twist.h"
#include "nav_msgs/Odometry.h"
#include "sensor_msgs/LaserScan.h"
#include "std_msgs/Float32.h"
#include "std_msgs/Int16.h"
#include "unistd.h"
#include <ros/ros.h>

#include <list>
#include <string>

using namespace std;

// RosbotClass constructor
RosbotClass::RosbotClass() {
  n = ros::NodeHandle("~");
  laser_topic = "/scan";
  laser_sub = n.subscribe(laser_topic, 10, &RosbotClass::laser_callback, this);
  vel_topic = "/cmd_vel";
  vel_pub = n.advertise<geometry_msgs::Twist>(n.resolveName(vel_topic), 1);
  odom_topic = "/odom";
  odom_sub = n.subscribe(odom_topic, 10, &RosbotClass::odom_callback, this);
  ROS_INFO("Initializing node .................................");
  usleep(2000000);
}

void RosbotClass::laser_callback(
    const sensor_msgs::LaserScan::ConstPtr &laser_msg) {
  laser_range = laser_msg->ranges;
  // ROS_INFO("Laser value: %f", laser_range);
}
        
void RosbotClass::odom_callback(const nav_msgs::Odometry::ConstPtr &odom_msg) {
  x_pos = odom_msg->pose.pose.position.x;
  y_pos = odom_msg->pose.pose.position.y;
  z_pos = odom_msg->pose.pose.position.z;
  // ROS_INFO_STREAM("Odometry: x=" << x_pos << " y=" << y_pos << " z=" <<
  // z_pos);
}
        
void RosbotClass::move() {
  // Rate of publishing
  ros::Rate rate(10);

  ros::Time start_time = ros::Time::now();
  ros::Duration timeout(2.0); // Timeout of 2 seconds
  while (ros::Time::now() - start_time < timeout) {
    ros::spinOnce();
    vel_msg.linear.x = +0.5;
    vel_msg.angular.z = 0.0;
    vel_pub.publish(vel_msg);
    rate.sleep();
  }
  vel_msg.linear.x = 0.0;
  vel_msg.angular.z = 0.0;
  vel_pub.publish(vel_msg);
}

void RosbotClass::move_forward(int time) {
  // Rate of publishing
  ros::Rate rate(10);

  ros::Time start_time = ros::Time::now();
  ros::Duration timeout(time);
  while (ros::Time::now() - start_time < timeout) {
    ROS_INFO_STREAM("Moving forward ........... ");
    ros::spinOnce();
    vel_msg.linear.x = 0.4;
    vel_msg.angular.z = 0.0;
    vel_pub.publish(vel_msg);
    rate.sleep();
  }
  vel_msg.linear.x = 0.0;
  vel_msg.angular.z = 0.0;
  vel_pub.publish(vel_msg);
}

void RosbotClass::move_backwards(int time) {
  // Rate of publishing
  ros::Rate rate(10);

  ros::Time start_time = ros::Time::now();
  ros::Duration timeout(time);
  while (ros::Time::now() - start_time < timeout) {
    ROS_INFO_STREAM("Moving backwards ........... ");
    ros::spinOnce();
    vel_msg.linear.x = 0.5;
    vel_msg.angular.z = 0.0;
    vel_pub.publish(vel_msg);
    rate.sleep();
  }
  vel_msg.linear.x = 0.0;
  vel_msg.angular.z = 0.0;
  vel_pub.publish(vel_msg);
}

void RosbotClass::turn(string clock, int n_secs) {
  ros::Rate rate(10);
  ros::Time start_time = ros::Time::now();
  ros::Duration timeout(n_secs);

  double WZ = 0.0;
  if (clock == "clockwise") {
    ROS_INFO_STREAM("Turning clockwise..............");
    WZ = -2.5;
  } else if (clock == "counterclockwise") {
    ROS_INFO_STREAM("Turning counterclockwiseeee ........... ");
    WZ = 2.5;
  }

  while (ros::Time::now() - start_time < timeout) {
    ros::spinOnce();
    vel_msg.linear.x = 0.5;
    vel_msg.angular.z = WZ;
    vel_pub.publish(vel_msg);
    rate.sleep();
  }
  vel_msg.linear.x = 0.0;
  vel_msg.angular.z = 0.0;
  vel_pub.publish(vel_msg);
}

void RosbotClass::stop_moving() {
  ROS_INFO_STREAM("Stopping the robot ........... ");
  vel_msg.linear.x = 0.0;
  vel_msg.angular.z = 0.0;
  vel_pub.publish(vel_msg);
}

float RosbotClass::get_position(int param) {
  if (param == 1) {
    return this->x_pos;
  } else if (param == 2) {
    return this->y_pos;
  } else if (param == 3) {
    return this->z_pos;
  }
  return 0;
}

list<float> RosbotClass::get_position_full() {
  list<float> coordinates({this->x_pos, this->y_pos, this->z_pos});
  return coordinates;
}

double RosbotClass::get_time() {
  double secs = ros::Time::now().toSec();
  return secs;
}

float RosbotClass::get_laser(int index) { return laser_range[index]; }

float *RosbotClass::get_laser_full() {
  float *laser_range_pointer = laser_range.data();
  return laser_range_pointer;
}

int main(int argc, char **argv) {
  ros::init(argc, argv, "rosbot_class_node");

  RosbotClass rosbot;

  rosbot.move();

  float coordinate = rosbot.get_position(1);

  ROS_INFO_STREAM(coordinate);

  return 0;
}

This code defines a class called RosbotMove which includes an instance of RosbotClass, and two member functions: get_out() and calc_distance().

The get_out() function is responsible for moving the robot out of a starting location, which involves driving forward until the robot detects an obstacle using its frontal laser sensor. The robot then turns clockwise, drives forward for a bit, turns counterclockwise, and moves forward until it has travelled at least 8 units. Finally, it turns clockwise and moves forward for 5 units before printing a success message.

The calc_distance() function calculates the Euclidean distance between two points given their (x, y) coordinates.

The main() function initializes a RosbotMove object and calls its get_out() function.

In [ ]:
#include "rosbot_control/rosbot_class.h"
#include <ros/ros.h>

#include <math.h>
#include <string>

using namespace std;

class RosbotMove {
public:
  RosbotClass rosbot;
  void get_out();
  float calc_distance(float x0, float y0, float x1, float y1);
};

void RosbotMove::get_out() {
  rosbot.move_forward(1);
  while (rosbot.get_laser(0) > 1.75) {
    ROS_INFO_STREAM("Laser frontal reading: " << rosbot.get_laser(0));
    rosbot.move_forward(1);
  }
  rosbot.turn("clockwise", 3);
  rosbot.move_forward(2);
  rosbot.turn("counterclockwise", 3);

  // Get initial position
  float x_0 = rosbot.get_position(1);
  float y_0 = rosbot.get_position(2);
  float x_1 = x_0;
  float y_1 = y_0;
  float dist = calc_distance(x_0, y_0, x_1, y_1);
  while (dist < 8.00) {
    // Update current position
    x_1 = rosbot.get_position(1);
    y_1 = rosbot.get_position(2);
    dist = calc_distance(x_0, y_0, x_1, y_1);
    ROS_INFO_STREAM("Distance travelled: " << dist);
    // Keep moving
    rosbot.move_forward(1);
  }
  rosbot.turn("clockwise", 3);
  rosbot.move_forward(5);
  ROS_INFO_STREAM("Success!!!");
}

    
// function to calculate distance
float RosbotMove::calc_distance(float x0, float y0, float x1, float y1)
{
    // calculating distance
    return sqrt(pow(x1 - x0, 2) + pow(y1 - y0, 2));
}

int main(int argc, char **argv) {
  ros::init(argc, argv, "Rosbot_move_node");
  RosbotMove rosbot_moves;
  rosbot_moves.get_out();
}